# Building a Conversational Chatbot using Rasa and Python

<b>Issue</b>: In many cases, Clients do not want to share their data and since  majority of the avialable tools are cloud-based and provide software as a service so you can not run them internally in your environment and you need to send your data to the third party. 

<b>Solution</b> : With <b>RASA stack</b> an open-source customizable AI tool there is no such issue. You can build, deploy or host Rasa internally in your server or environment with complete control on it.

Rasa comes up with 2 components —

<b>Rasa NLU:</b> NLU deals with teaching a chatbot on how to understand user inputs.<br>
<b>Rasa Core:</b> Deals with teaching a chatbot on how to respond to user’s query.

## Starting Jupyter Notebook with necessary imports

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

# Installations
* Rasa NLU
* Rasa Core
* SpaCy Language Model

In [ ]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -U rasa_core==0.9.6 rasa_nlu[spacy];

In [ ]:
!pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.12.0-py3-none-any.whl
!pip install -U rasa_core

In [ ]:
!{python} -m spacy download en_core_web_md

## Downloading the English Language Model

In [5]:
!{python} -m spacy link en_core_web_md en --force;

✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_md -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


# Importing the Installations

In [ ]:
!{python} -m pip install rasa_core==0.9.6 rasa_nlu

In [8]:
import rasa_nlu
import rasa_core
import spacy

#import rasa

In [ ]:
from rasa_core.agent import Agent

# 1. Teaching the bot to understand user inputs using Rasa NLU

## Preparing the NLU Training Data - Defining the intent
Intent is understanding what the user wants to say. For example — if the user says “Reserve a table at Cliff House tonight” the intent can be classified as to reserve/book the table.

In [8]:
nlu_md = """

## intent:greet
- hey
- hello
- hi
- Hi

## intent:fine_ask
- I am good, how are you doing?
- I'm fine, how are you?
- I'm good, how are you?

## intent:my_name_is
- I am [Atul](PERSON)
- I am [Sampriti](PERSON)
- I'm [Prerna](PERSON)
- im [Varun](PERSON)
- My name is [Nikhil](PERSON)

## intent:fine_normal
- I am doing great
- I'm doing great
- I'm fine
- I'm good

## intent:news
- Share some latest news around the [world](category)?
- Share some latest news in [sports](category)?
- What is going on in [technology](category)?
- Tell me some news about [fashion](category)
- Tell me some news about [business](category)
- Tell me some news about [arts](category)
- What is going on in [arts](category)
- What is cooking in [food](category)
- [movies](category)

## intent:thanks
- Thanks
- Thank you so much

## intent:bye
- No, I am good as of now. Bye
- Bye
- Bbye
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


## Defining the NLU Model Configuration

In [9]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


## Training the NLU Model.

In [10]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 743 kB 5.1 MB/s 


In [ ]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

In [12]:
%pwd

'/content'

## Evaluating the NLU model on a random text

In [13]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("Can you share some news from around the world? "))

{
  "intent": {
    "name": "news",
    "confidence": 0.555918269226017
  },
  "entities": [
    {
      "start": 40,
      "end": 45,
      "value": "world",
      "entity": "category",
      "confidence": 0.7921448143869898,
      "extractor": "CRFEntityExtractor"
    }
  ],
  "intent_ranking": [
    {
      "name": "news",
      "confidence": 0.555918269226017
    },
    {
      "name": "bye",
      "confidence": 0.12691241615979648
    },
    {
      "name": "fine_ask",
      "confidence": 0.1021956327336667
    },
    {
      "name": "thanks",
      "confidence": 0.08937229907831164
    },
    {
      "name": "my_name_is",
      "confidence": 0.06323478314948004
    },
    {
      "name": "fine_normal",
      "confidence": 0.03883600503898876
    },
    {
      "name": "greet",
      "confidence": 0.02353059461373988
    }
  ],
  "text": "Can you share some news from around the world? "
}


## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [28]:
from rasa_nlu.test import run_evaluation

run_evaluation("nlu.md", model_directory)

INFO:rasa_nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 30 (7 distinct intents)
	- Found intents: 'fine_ask', 'thanks', 'greet', 'my_name_is', 'bye', 'fine_normal', 'news'
	- entity examples: 14 (2 distinct entities)
	- found entities: 'category', 'PERSON'

INFO:rasa_nlu.test:Running model for predictions:
100%|██████████| 30/30 [00:00<00:00, 146.79it/s]
INFO:rasa_nlu.test:Intent evaluation results:
INFO:rasa_nlu.test:Intent Evaluation: Only considering those 30 examples that have a defined intent out of 30 examples
INFO:rasa_nlu.test:F1-Score:  1.0
INFO:rasa_nlu.test:Precision: 1.0
INFO:rasa_nlu.test:Accuracy:  1.0
INFO:rasa_nlu.test:Classification report: 
              precision    recall  f1-score   support

         bye       1.00      1.00      1.00         3
    fine_ask       1.00      1.00      1.0

{'entity_evaluation': {'CRFEntityExtractor': {'accuracy': 1.0,
   'f1_score': 1.0,
   'precision': 1.0,
   'report': '              precision    recall  f1-score   support\n\n      PERSON       1.00      1.00      1.00         5\n    category       1.00      1.00      1.00         9\n   no_entity       1.00      1.00      1.00       114\n\n   micro avg       1.00      1.00      1.00       128\n   macro avg       1.00      1.00      1.00       128\nweighted avg       1.00      1.00      1.00       128\n'}},
 'intent_evaluation': {'accuracy': 1.0,
  'f1_score': 1.0,
  'precision': 1.0,
  'predictions': [{'confidence': 0.5701962932622745,
    'intent': 'greet',
    'predicted': 'greet',
    'text': 'hey'},
   {'confidence': 0.5808877030589578,
    'intent': 'greet',
    'predicted': 'greet',
    'text': 'hello'},
   {'confidence': 0.5931896748718971,
    'intent': 'greet',
    'predicted': 'greet',
    'text': 'hi'},
   {'confidence': 0.5931896748718971,
    'intent': 'greet',
    'predic

# 2. Teaching the bot to respond using Rasa Core

## 1. Writing  Stories

<b>stories_md</b> define the sample interaction between the user and chatbot in terms of intent and action taken by the bot. Like in the previous example bot got the intent of booking the table and entities like place and time but still, there is an entity missing — no of people and that would make the next action from the bot.

In [29]:
stories_md = """
## fallback
- utter_default

## greeting path 1
* greet
- utter_greet

## my name is
* my_name_is
- utter_its_nice_to_meet_you

## fine path 1
* fine_normal
- utter_help

## fine path 2
* fine_ask
- utter_reply

## news path
* news
- utter_ofc
- action_get_news

## thanks path 1
* thanks
- utter_anything_else

## bye path 1
* bye
- utter_bye
"""

%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


## 2. Defining a Domain
### Domain(domain.yml)
The domain consists of five key parts consisting of intents, entities, slots, actions, and templates.

<b>slots:</b> Slots are basically bot’s memory. They act as a key-value store which can be used to store information the user provided (e.g their home city) as well as information gathered about the outside world (e.g. the result of a database query).

<b>entity:</b> Entity is to extract the useful information from the user input. From the example above <i>“Reserve a table at Cliff House tonight”</i> the entities extracted would be place and time. Place — Cliff House and Time — tonight.

<b>intent: </b> Intent is understanding what the user wants to say. For example — if the user says <i>“Reserve a table at Cliff House tonight”</i> the intent can be classified as to reserve/book the table.

<b>actions:</b> Actions are bots response to user input. There are 3 kinds of actions in Rasa Core: <b>default actions, utter actions & custom actions</b>

<b>templates:</b> templates define the actual text responses used by the dialogue engine. The engine will pick one random response out of all the options. Notice that <b>utter_its_nice_to_meet_you</b> uses <b>PERSON</b> slot in the response to personalize it.

In [16]:
domain_yml = """
slots:
  PERSON:
    type: text

entities:
- category

intents:
- greet
- fine_ask
- fine_normal
- news
- thanks
- bye

actions:
- action_restart
- action_get_news
- utter_greet
- utter_reply
- utter_help
- utter_anything_else
- utter_ofc
- utter_bye
- utter_default
- utter_its_nice_to_meet_you

templates:

  utter_greet:
    - text: Hey {PERSON}, how are you?
    - text: Hello {PERSON}, How are you doing?
    
  utter_its_nice_to_meet_you:
    - It's nice to meet you, {PERSON}.
    - Nice to meet you, {PERSON}.
  
  utter_reply:
    - text: I'm doing great. Please let me know what I can do for you.
    - text: I'm doing great. Tell me How can I help you today?
  utter_help:
    - text: Great{PERSON}. How can I help you?
    - text: Great. Tell me How can I help you?
    - text: Great. Tell me what all news you would like to get.
  utter_anything_else:
    - text: No worries. Is there anything else I can help you with?
    - text: No worries. Let me know if there is anything else I can help you with
  utter_ofc:
    - text: I can definitely help you. The top 5 news of the {category}
    - text: Surely, I can help you. The top 5 news of the {category}
  utter_bye:
    - text: Bye and have a nice day
    - text: Bbye and have a nice day
  utter_default:
    - text: I am not sure what you're aiming for
    - text: I am sorry but I am not able to get you.
    - text: My appologies but I am not able to get you
"""


%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


## Policies - policiy.yml

The rasa core policies decide which action to take at every step in the conversation. There are different policies to choose from, and one can include multiple policies in a single rasa core Agent. But at every turn, the policy which predicts the next action with the highest confidence is used. We have configured a basic policy <b>(policy.yml)</b> for our bot as shown below which has <b>FallbackPolicy</b> as well. The fallback policy comes in to picture when <b>‘nlu_threshold’ & ‘core_threshold’</b> meets the levels defined in the policy which means that bot is not able to understand the user message and it responds with <b>‘utter_default’</b>.

<b>KerasPolicy</b> uses a neural network implemented in Keras to select the next action. The default architecture is based on an LSTM (Long Short Term Memory) model

<b>MemoizationPolicy</b> memorizes the conversations in your training data. It predicts the next action with confidence 1.0 if this exact conversation exists in the training data, otherwise, it predicts ‘None’ with confidence 0.0

<b>FallbackPolicy</b> invokes a fallback action if the intent recognition has confidence below nlu_threshold or if none of the dialogue policies predict action with confidence higher than core_threshold

One important hyperparameter for Rasa Core policies is the <b>max_history</b>. This controls how much dialogue history the model looks at to decide which action to take next

In [17]:
policy_yml = """

policies:
  - name: KerasPolicy
    epochs: 1000
  - name: FallbackPolicy
    fallback_action_name: 'utter_default'
    nlu_threshold: 0.1
    core_threshold: 0.2
  - name: MemoizationPolicy
    max_history: 5
    
  - name: FormPolicy
    """
%store policy_yml > policies.yml

Writing 'policy_yml' (str) to file 'policies.yml'.


## Custom Actions

In [18]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.core.display import Image, display

from rasa_core_sdk import Action
import requests
import json


class ActionGetNewst(Action):

    def name(self):
        return 'action_get_news'

    def run(self, dispatcher, tracker, domain):
        category = tracker.get_slot('category')
        print(category)
        
        url = 'https://api.nytimes.com/svc/news/v3/content/all/{category}.json'.format(category=category)
        params = {'api-key': "2hq54bvFO0yWiRdY70reBU2GmusBtnwM", 'limit': 5}
        response = requests.get(url, params).text
        json_data = json.loads(response)['results']
        i = 0
        for results in json_data:
            i = i+1
            message = str(i) + "." + results['abstract']
            dispatcher.utter_message(message)
        return[]

##  Visualising the Training Data

In [ ]:
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
!breq install graphviz

!{python} -m pip install pygraphviz;

In [56]:
from IPython.display import Image
import rasa_core.test

In [57]:
rasa_core.test()
# # async load_from_server(agent: "Agent", model_server: EndpointConfig) -> "Agent"
# create_agent(model: Text, endpoints: Text = None) -> "Agent"

TypeError: ignored

In [ ]:
agent = Agent('domain.yml')
agent.visualize("stories.md", "story_graph.png", max_history=2)
Image(filename="story_graph.png")

## Training a Dialogue Model

In [ ]:
from rasa_core import config as policy_config
from rasa_core.agent import Agent

from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
#from rasa_core.agent import Agent
# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions

fallback = FallbackPolicy(nlu_threshold=0.1,core_threshold=0.2)
policies = policy_config.load("policies.yml")
agent = Agent("domain.yml", policies = policies)

#agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])
#agent = Agent('domain.yml',  policies=[MemoizationPolicy(max_history=5),KerasPolicy(epochs=100)])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(training_data)

#validation_split=0.2


# agent.train(
#     training_data,
#     validation_split=0.0,
#     epochs=200
# )

agent.persist('/home/edureka/models/dialogue')

# Talk to your Bot

In [ ]:
#Starting the Bot

from rasa_core.agent import Agent
agent = Agent.load('/home/edureka/models/dialogue', interpreter=model_directory)

In [54]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])     

Your bot is ready to talk! Type your messages here or send 'stop'
Good Day


NameError: ignored